In [5]:
!pip install dash
!pip install jupyter-dash
!pip install pandas
!pip install plotly
!pip install wget

In [6]:
from dash import Dash
from dash import dcc
from dash import html
from dash import Input
from dash import Output
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px
import wget

In [65]:
spacex_csv_file = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()

100% [............................................................] 2476 / 2476

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [66]:
def outcome(success_failed):
    if success_failed== 1:
        return 'Success'
    else:
        return 'Failed'   

spacex_df['Outcome'] = spacex_df['class'].apply(outcome)
spacex_df['Count'] = 1

min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()


In [78]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign':'center'}),
    dcc.Dropdown(id='site_dropdown',
                options=[
                    {'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                ],
                value='ALL',
                placeholder="Select a Launch Site here",
                searchable=True
                ),
    dcc.Graph(id='success_pie_chart'),
    dcc.RangeSlider(id='payload_slider',
                min=0, 
                max=10000,
                step=1000,
                marks={0: '0',
                       100: '100'},
                value=[min_payload, max_payload]),
    dcc.Graph(id='success_payload_scatter_chart')
    ])

@app.callback(
    Output(component_id='success_pie_chart', component_property='figure'),
    Input(component_id='site_dropdown', component_property='value')
    ) 
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        figure = px.pie(spacex_df, 
            values='class', 
            names='Launch Site', 
            title='Launch Success Rate For All Sites')
        return figure
    else:
        # return the outcomes piechart for a selected site
        entered_site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        figure = px.pie(entered_site_df,
            values='Count',
            names='Outcome',
            title='Launch Success Rate For ' + entered_site)
        return figure
    
@app.callback(
    Output(component_id='success_payload_scatter_chart', component_property='figure'),
    [Input(component_id='site_dropdown', component_property='value'), 
    Input(component_id='payload_slider', component_property='value')]
    )
def get_scatter_chart(entered_site, payload_slider):
    filtered_df = spacex_df[
        (payload_slider[0] <= spacex_df['Payload Mass (kg)']) & (spacex_df['Payload Mass (kg)'] <= payload_slider[1])
    ]
    if entered_site == 'ALL':
        figure = px.scatter(filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Launch Success Rate For All Sites')
        return figure
    # return the outcomes in pie chart for a selected site
    else:
        entered_site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        figure = px.scatter (entered_site_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Launch Success Rate For ' + entered_site)
    return figure

In [79]:
if __name__ == '__main__':
    app.run_server()

Dash app running on http://127.0.0.1:8050/
